# OpenAI Agent with LlamaIndex

## Install Dependencies

In [ ]:
# !pip install uv
# !uv pip install -qU llama-index-llms-openai llama-index-readers-file llama-index-embeddings-openai "openinference-instrumentation-llama-index>=2" arize-phoenix python-dotenv

I0000 00:00:1750537025.539120 4407791 fork_posix.cc:71] Other threads are currently calling into gRPC, skipping fork() handlers


I0000 00:00:1750537026.472321 4407791 fork_posix.cc:71] Other threads are currently calling into gRPC, skipping fork() handlers


## Setup API Keys


In [1]:
from os import environ
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = environ["OPENAI_API_KEY"]

## Import libraries and setup LlamaIndex

In [2]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI


# Create an llm object to use for the QueryEngine and the ReActAgent
llm = OpenAI(model="gpt-4")

In [3]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI


# Create an llm object to use for the QueryEngine and the ReActAgent
llm = OpenAI(model="gpt-4")

# Set up Phoenix

In [4]:
import phoenix as px
session = px.launch_app()

/Users/j.ning/Agent-Hack/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/homebrew/Cellar/python@3.12/3.12.11/Frameworks/Python.framework/Versions/3.12/lib/python3.12/contextlib.py:144: SAWarning: Skipped unsupported reflection of expression-based index ix_cumulative_llm_token_count_total
  next(self.gen)
/opt/homebrew/Cellar/python@3.12/3.12.11/Frameworks/Python.framework/Versions/3.12/lib/python3.12/contextlib.py:144: SAWarning: Skipped unsupported reflection of expression-based index ix_latency
  next(self.gen)


🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://arize.com/docs/phoenix


In [5]:
from openinference.instrumentation.llama_index import LlamaIndexInstrumentor
from phoenix.otel import register

tracer_provider = register()
LlamaIndexInstrumentor().instrument(tracer_provider=tracer_provider)

🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: default
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: localhost:4317
|  Transport: gRPC
|  Transport Headers: {'user-agent': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  ⚠️ WARNING: It is strongly advised to use a BatchSpanProcessor in production environments.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



## Load Documents

In [ ]:
try:
    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/custom"
    )
    lyft_index = load_index_from_storage(storage_context)

    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/uber"
    )
    uber_index = load_index_from_storage(storage_context)

    index_loaded = True
except:
    index_loaded = False

Loading llama_index.core.storage.kvstore.simple_kvstore from ./storage/lyft/docstore.json.


This is the point we create our vector indexes, by calculating the embedding vectors for each of the chunks. You only need to run this once.

In [ ]:
if not index_loaded:
    # load data
    custom_rules_docs = SimpleDirectoryReader(
        input_files=["./constant/custom_rules_1.0.md"]
    ).load_data()
    scanning_result_docs = SimpleDirectoryReader(
        input_files=["./constant/qualys_sample_report.json"]
    ).load_data()

    # build index
    custom_index = VectorStoreIndex.from_documents(custom_rules_docs, show_progress=True)
    scan_index = VectorStoreIndex.from_documents(scanning_result_docs, swow_progress=True)

    print(custom_rules_docs)
    # persist index
    custom_index.storage_context.persist(persist_dir="./constant/custom_rules")
    scan_index.storage_context.persist(persist_dir="./constant/scan_results")

Generating embeddings: 100%|██████████| 1/1 [00:00<00:00,  2.03it/s]


[Document(id_='f4be0c74-1a72-4735-84bc-358d3a3a22ed', embedding=None, metadata={'file_path': 'constant/custom_rules_1.0.md', 'file_name': 'custom_rules_1.0.md', 'file_type': 'text/markdown', 'file_size': 4062, 'creation_date': '2025-06-21', 'last_modified_date': '2025-06-21'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='\n# 🔒 Security Vulnerability Handling Rules\n\n## 1. 📊 General Post-Processing by Severity\n\n| Severity Level | Action |\n|----------------|--------|\n| **High** | File a **bug ticket** (severity: **High**).<br>Send an **email notification** to the responsible team’s **engineering manager** and **security contact**.<br

Now create the query engines.

In [14]:
custom_engine = custom_index.as_query_engine(similarity_top_k=3, llm=llm)
scan_engine = uber_index.as_query_engine(similarity_top_k=3, llm=llm)

We can now define the query engines as tools that will be used by the agent.

As there is a query engine per document we need to also define one tool for each of them.

In [17]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=custom_engine,
        metadata=ToolMetadata(
            name="custom_rules",
            description=(
                "Provides information about post process of vulnerability scan report. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=scan_engine,
        metadata=ToolMetadata(
            name="scanning_results",
            description=(
                "Provides information about qualys vulnerability reports. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
]

## Creating the Agent
Now we have all the elements to create a LlamaIndex ReactAgent

In [18]:
agent = ReActAgent.from_tools(
    query_engine_tools,
    llm=llm,
    verbose=True,
    max_turns=10,
)

Now we can interact with the agent and ask a question.

In [24]:
response = agent.chat("You are the security engineer, you have 3 actions based on the custom rules file. You could either ignore the vulnerability, email the respective team, and file a bug with the right information. Read the first case of scan result, what's the next step and take action in plain text")
print(str(response))

Transient error StatusCode.UNAVAILABLE encountered while exporting traces to localhost:4317, retrying in 1s.


> Running step 5b6132d4-9a88-4cd2-8f82-28fce2542581. Step input: You are the security engineer, you have 3 actions based on the custom rules file. You could either ignore the vulnerability, email the respective team, and file a bug with the right information. Read the first case of scan result, what's the next step and take action in plain text
Thought: The user is asking for the next step based on the first case of the scan result. To provide an accurate response, I need to use the scanning_results tool to get the first case of the scan result.
Action: scanning_results
Action Input: {'input': 'What is the first case of the scan result?'}
Observation: The first case of the scan result is for the asset with IP "10.0.1.15" and hostname "web-01.example.internal". The operating system of this asset is "Ubuntu 22.04". It has a vulnerability titled "OpenSSH 'Channel' double-free privilege escalation" with a severity of 5 and a CVSS base score of 9.8. This vulnerability, identified as "CVE-20